# Linear Graph

In [14]:
def function_1(input):
    return input + " Hi "

def function_2(input):
    return input + "there"

def function_3(input):
    if input == "Hello Hi":
        return "Next"
    else:
        return input_2 + "there"

def function_4(input):
    return "Agent says :" +input 

In [15]:
from langgraph.graph import Graph

# Define a Langchain graph
workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('node_1', 'node_2')

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [16]:
input = 'Hello'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'node_1':
---
Hello Hi 

---

Output from node 'node_2':
---
Hello Hi there

---



# Decision Graph

In [8]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [9]:
def function_1(state):
    messages = state['messages']
    message =  "Hi"
    return {"messages": [message]} 

def function_2(state):
    messages = state['messages']
    message = "there"
    return {"messages": [message]} 

def function_3(state):
    messages = state['messages']
    for i in state['messages']:
        print(i)
    if messages[0]=="Hello":
        return "node2"
    else:
        return "node4"
    return

def function_4(state):
    messages = state['messages']
    message = "Agent"
    return {"messages": [message]}  

In [10]:
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)
workflow.add_node("node_4", function_4)
workflow.add_conditional_edges("node_1", function_3,{   # Based on the return from function_3
                                                        # If return is "continue" then we call the tool node.
                                                        "node2": "node_2",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "node4": "node_4"
                                                    }
)


workflow.add_edge('node_2',END)
workflow.add_edge('node_4',END)
workflow.set_entry_point("node_1")


app = workflow.compile()

In [11]:
inputs = {"messages": ["Joy"]}
app.invoke(inputs)

Joy
Hi


{'messages': ['Joy', 'Hi', 'Agent']}

In [13]:
inputs = {"messages": ["Hello"]}
app.invoke(inputs)

Hello
Hi


{'messages': ['Hello', 'Hi', 'there']}

In [12]:
inputs = {"messages": ["Hello"]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Hello
Hi
Output from node 'node_1':
---
{'messages': ['Hi']}

---

Output from node 'node_2':
---
{'messages': ['there']}

---

